# Exploring data

When working with a dataset, the first step is usually to understand what data and metadata it contains.
In this chapter we explore how Scipp supports this.

This tutorial contains exercises, but solutions are included directly.
We encourage you to download this notebook and run through it step by step before looking at the solutions.
We recommend to use a recent version of *JupyterLab*:
The solutions are included as hidden cells and shown only on demand.

First, in addition to importing `scipp`, we import `scippneutron` for neutron-science routines.

In [ ]:
import scipp as sc
import scippneutron as scn
import plopp as pp

We  start by loading some data, in this case measured with a prototype of the [LoKI](https://europeanspallationsource.se/instruments/loki) detectors at the [LARMOR beamline](https://www.isis.stfc.ac.uk/Pages/Larmor.aspx):

In [ ]:
data = scn.data.tutorial_dense_data()

In practice, we would use [scippneutron.load_with_mantid](../generated/functions/scippneutron.load_with_mantid.rst) or [scippneutron.load_nexus](../generated/functions/scippneutron.load_nexus.rst) to load the data from a NeXus file, but the tutorial data comes bundled with ScippNeutron to make it easily available.
See [Tutorial and Test Data](../developer/getting-started.md#tutorial-and-test-data) for a way to customize where the data is stored.

Note that the exercises in the following are fictional and do not represent the actual SANS data reduction workflow.

## Step 1: Use the HTML representation to see what the loaded data contains

The HTML representation is what Jupyter displays for a scipp object.

- Take some time to explore this view and try to understand all the information (dimensions, dtypes, units, ...).
- Note that sections can be expanded, and values can shown by clicking the icons to the right.

In [ ]:
data

In [ ]:
detector = data['detector']
detector

Whenever you get stuck with one of the exercises below we recommend consulting the HTML representations of the objects you are working with.

## Step 2: Plot the data

Scipp objects (variables, data arrays, datasets, or data groups) can be plotted using the `plot()` method.
Alternatively `sc.plot(obj)` can be used, e.g., when `obj` is a Python `dict` of scipp data arrays.
Since this is neutron-scattering data, we can also use the "instrument view", provided by `scn.instrument_view(obj)` (assuming `scippneutron` was imported as `scn`).

### Exercise

- Plot the loaded data and familiarize yourself with the controls.
- Create the instrument view and familiarize yourself with the controls.

### Solution

In [ ]:
detector.sum('spectrum').plot()

In [ ]:
scn.instrument_view(detector.sum('tof'), norm='log')

## Step 3: Exploring metadata

Above we saw that the input data group contains a number of metadata items in addition to the the main 'detector'.
Some items are simple strings while others are data arrays or variables.
These have various meanings and we now want to explore them.

### Exercise

1. Find some data array items of `data` and plot them.
   Also try `sc.table(item)` to show a table representation (where `item` is an item of your choice).
2. Find and plot a monitor.
3. Try to normalize `detector` to monitor 1.
   Why does this fail?
4. Plot all the monitors on the same plot.
   Note that `sc.plot()` can be used with a data group.
5. Convert all the monitors from `'tof'` to `'wavelength'` using, e.g.,
   ```python
   wavelength_graph_monitor = {
       **scn.conversion.graph.beamline.beamline(scatter=False),
       **scn.conversion.graph.tof.elastic_wavelength('tof'),
   }
   mon1_wav = mon1.transform_coords('wavelength', graph=wavelength_graph_monitor)
   ```
7. Inspect the HTML view and note how the "unit conversion" changed the dimensions and units.
8. Re-plot all the monitors on the same plot, now in `'wavelength'`.

### Solution

In [ ]:
sc.table(data['DCMagField2'])

In [ ]:
detector / data['monitors']['monitor1']

A monitor can be converted to wavelength as follows:

In [ ]:
wavelength_graph_monitor = {
    **scn.conversion.graph.beamline.beamline(scatter=False),
    **scn.conversion.graph.tof.elastic_wavelength('tof'),
}

In [ ]:
mon1 = data['monitors']['monitor1']
mon1.transform_coords('wavelength', graph=wavelength_graph_monitor)

In [ ]:
sc.plot(data['monitors'], norm='log')

In [ ]:
converted_monitors = {
    f'monitor{i}': data['monitors'][f'monitor{i}'].transform_coords(
        'wavelength', graph=wavelength_graph_monitor
    )
    for i in [1, 2, 3, 4, 5]
}
sc.plot(converted_monitors, norm='log')

## Step 4: Fixing metadata

### Exercise

Consider the following (hypothetical) problems with the metadata stored in `detector`:

1. The `sample_position` coord (`detector.coords['sample_position']`) is wrong, shift the sample by `delta = sc.vector(value=np.array([0.01,0.01,0.04]), unit='m')`.
2. Because of a glitch in the timing system the time-of-flight has an offset of $2.3~\mu s$.
   Fix the corresponding coordinate.
3. Use the HTML view of `detector` to verify that you applied the corrections/calibrations there, rather than in a copy.

### Solution

In [ ]:
detector.coords['sample_position'] += sc.vector(value=[0.01, 0.01, 0.04], unit='m')
detector.coords['tof'] += 2.3 * sc.Unit(
    'us'
)  # note how we forgot to fix the monitor's TOF
detector

Note how adding such offsets fails if we fail to specify a unit:

In [ ]:
detector.coords['tof'] += 2.3

This has several advantages:
- We are protected from accidental errors.
  If someone changes the unit of data or metadata without our knowledge, e.g., from `us` to `ms` this mechanism protects us from silent errors corrupting the data.
- It makes the code clearer and more readable, both for others as well as for our future selves.

The offset to the sample could also be done component-wise using the special `fields` property of variables with "vector" dtype, e.g.,

In [ ]:
detector.coords['sample_position'].fields.z += 0.001 * sc.Unit('m')

## Step 5: A closer look at the data

In the very first task, we summed the data along the `spectrum` dimension before plotting,
yielding a pixel-integrated time-of-flight spectrum of the data.
Here, we sum along the `tof` dimension to inspect the contents of individual pixels:

In [ ]:
counts = detector.sum('tof')

### Exercise

1. Create a plot of `counts` and also try the instrument view.
2. How many counts are there in total, in all spectra combined?
3. Plot a single spectrum of `detector` as a 1-D plot using the slicing syntax `array[dim_name, integer_index]` to access the spectrum.

### Solution

In [ ]:
# slice is optional, making plot more readable in the documentation
counts['spectrum', 56000:62000].plot()

In [ ]:
scn.instrument_view(counts, norm='log')

In [ ]:
# detector.sum('spectrum') # would be another solution
detector.sum().value

In [ ]:
detector['spectrum', 10000].plot()

### Volumetric detectors

As seen in the instrument view the detectors consist of 4 layers of tubes, each containing 7 straws.
Let us try to split up our data, so we can compare layers.
There are other (and probably better) ways to do this, but here we try to define an integer variable containing a layer index:

In [ ]:
z = detector.coords['position'].fields.z
near = z.min()
far = z.max()
layer = ((z - near) * 400).astype('int32')
layer.unit = ''
layer.plot()

### Exercise

- Change the magic parameter `400` in the cell above until pixels fall cleanly into layers, either 4 layers of tubes or 12 layers of straws.
- Store `layer` as a new coord in `detector`.
- Use `detector.groupby(group='layer').sum('spectrum')` to group spectra into layers.
- Inspect and understand the HTML view of the result.
- Plot the result.
  There are two options:
  - Use [Plopp's `slicer` utility](https://scipp.github.io/plopp/basics/slicer-plot.html) to navigate the different layers using a slider (requires `%matplotlib widget` to enable interactive figures)
  - Use `sc.plot` after collapsing dimensions, `sc.collapse(grouped, keep='tof')`
- Bonus: When grouping by straw layers, there is a different number of straws in the center layer of each tube (3 instead of 2) due to the flower-pattern arrangement of straws.
  Define a helper data array with data set to 1 for each spectrum (using, e.g., `norm = sc.DataArray(data=sc.ones_like(layer), coords={'layer':layer})`), group by layers and sum over spectrum as above, and use this result to normalize the layer-grouped data from above to spectrum count.

### Solution

In [ ]:
%matplotlib widget
# NOTE:
# - set magic factor to, e.g., 150 to group by straw layer
# - set magic factor to, e.g., 40 to group by tube layer
layer = ((z - near) * 150).astype(sc.DType.int32)
layer.unit = ''
detector.coords['layer'] = layer
grouped = detector.groupby(group='layer').sum('spectrum')
pp.slicer(grouped)

In [ ]:
sc.plot(sc.collapse(grouped, keep='tof'))

In [ ]:
norm = sc.DataArray(data=sc.ones_like(layer), coords={'layer': layer})
norm = norm.groupby(group='layer').sum('spectrum')
sc.plot(sc.collapse(grouped / norm, keep='tof'))